<h2>Library Preparation</h2>

In [1]:
import time
import os
import sys

import pandas as pd

from dotenv import load_dotenv

from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException

from webdriver_manager.chrome import ChromeDriverManager

from bs4 import BeautifulSoup

from lxml import etree

In [2]:
load_dotenv()

WAIT_DELAY = 10 #seconds

<h2>Open Browser and Instagram Url</h2>

In [3]:
def generateSeleniumOptions():
    options = Options()
    options.headless = True
    options.add_argument('--no-sandbox') # Bypass OS security model
    options.add_argument('start-maximized')
    options.add_argument('disable-infobars')
    options.add_argument("--disable-extensions")
    options.add_argument('user-agent={0}'.format('Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:100.0) Gecko/20100101 Firefox/100.0'))
    options.add_argument('--log-level=3')
    options.add_experimental_option('excludeSwitches', ['enable-logging'])
    return options

In [4]:
# driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=generateSeleniumOptions())
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))

[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 102.0.5005
[WDM] - Get LATEST chromedriver version for 102.0.5005 google-chrome
[WDM] - Driver [C:\Users\hapel\.wdm\drivers\chromedriver\win32\102.0.5005.61\chromedriver.exe] found in cache


In [5]:
URL="https://www.instagram.com/"
driver.get(URL)

<h2>Login to Instagram</h2>

In [6]:
username_xpath = "//*[@id='loginForm']/div/div[1]/div/label/input"
password_xpath = "//*[@id='loginForm']/div/div[2]/div/label/input"

login_button_xpath = "//*[@id='loginForm']/div/div[3]/button"

instagram_username = os.getenv('INSTAGRAM_USERNAME')
instagram_password = os.getenv('INSTAGRAM_PASSWORD')

try:
    WebDriverWait(driver, WAIT_DELAY).until(EC.presence_of_element_located((By.XPATH , username_xpath)))
except TimeoutException:
    print("timeout opening instagram web")
    raise

driver.find_element(By.XPATH , username_xpath).send_keys(instagram_username)
driver.find_element(By.XPATH , password_xpath).send_keys(instagram_password)
driver.find_element(By.XPATH , login_button_xpath).click()

<h2>Scrape instagram posts</h2>

In [7]:
not_now_xpath = '//*[@id="react-root"]/section/main/div/div/div/div/button'

try:
    WebDriverWait(driver, WAIT_DELAY).until(EC.presence_of_element_located((By.XPATH, not_now_xpath)))
except TimeoutException:
    print("timeout logging in to instagram")
    raise

driver.find_element(By.XPATH, not_now_xpath).click()

timeout logging in to instagram


TimeoutException: Message: 
Stacktrace:
Backtrace:
	Ordinal0 [0x011CD953+2414931]
	Ordinal0 [0x0115F5E1+1963489]
	Ordinal0 [0x0104C6B8+837304]
	Ordinal0 [0x01079500+1021184]
	Ordinal0 [0x0107979B+1021851]
	Ordinal0 [0x010A6502+1205506]
	Ordinal0 [0x010944E4+1131748]
	Ordinal0 [0x010A4812+1198098]
	Ordinal0 [0x010942B6+1131190]
	Ordinal0 [0x0106E860+976992]
	Ordinal0 [0x0106F756+980822]
	GetHandleVerifier [0x0143CC62+2510274]
	GetHandleVerifier [0x0142F760+2455744]
	GetHandleVerifier [0x0125EABA+551962]
	GetHandleVerifier [0x0125D916+547446]
	Ordinal0 [0x01165F3B+1990459]
	Ordinal0 [0x0116A898+2009240]
	Ordinal0 [0x0116A985+2009477]
	Ordinal0 [0x01173AD1+2046673]
	BaseThreadInitThunk [0x75806739+25]
	RtlGetFullPathName_UEx [0x771D8FEF+1215]
	RtlGetFullPathName_UEx [0x771D8FBD+1165]


In [ ]:
close_notification_xpath = '/html/body/div[5]/div/div/div/div[3]/button[2]'

try:
    WebDriverWait(driver, WAIT_DELAY).until(EC.presence_of_element_located((By.XPATH, close_notification_xpath)))
except TimeoutException:
    print("timeout logging in to instagram")
    raise

driver.find_element(By.XPATH, close_notification_xpath).click()

In [ ]:
def scroll_down():
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

In [ ]:
bs = BeautifulSoup(driver.page_source)
articles = bs.find_all("article", attrs={"role":"presentation"})
while True:
    articles.extend(bs.find_all("article", attrs={"role":"presentation"}))
    if len(articles) < 5:
        scroll_down()
    else:
        break

<h2>Proccess Data</h2>

In [ ]:
print(articles[0].prettify())

In [ ]:
username_class = "sqdOP yWX7d _8A5w5 ZIAjV"
image_class = "FFVAD"
like_class = "_7UhW9 xLCgt qyrsm KV-D4 fDxYl T0kll"

In [ ]:
posts = []
for article in articles:
    username = article.find("a", class_=username_class)
    image = article.find("img", class_=image_class)
    like_div = article.find_all("div", class_=like_class)
    print(like_div)
    post = {
        "username": username.text.strip(),
        "image": image['src'],
        "likes": likes
    }
    posts.append(post)

In [ ]:
post_df = pd.DataFrame(posts)
post_df

<h2>Close browser</h2>

In [ ]:
# driver.close()